<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">

# Derivatives Analytics with Python
**&mdash;Part II: Arbitrage Pricing**

&copy; Dr. Yves J. Hilpisch | The Python Quants

<a href="https://tpq.io" target="_blank">tpq.io</a> | <a href="https://linktr.ee/dyjh" target="_blank">linktr.ee/dyjh</a>

<img src="https://hilpisch.com/dawp_cover_small.png" width=30% align=left>

## Part II &mdash; Arbitrage Pricing

### Chapter 5 &mdash; Complete Market Models

This notebook introduces complete market models that make risk-neutral valuation fully
operational. The central example is the Black&mdash;Scholes&mdash;Merton model, which leads
to a closed-form valuation formula for European options and explicit expressions for the
main Greeks.

As a discrete-time counterpart, the Cox&mdash;Ross&mdash;Rubinstein binomial model provides an
intuitive tree-based valuation procedure and converges to the Black&mdash;Scholes&mdash;Merton
benchmark under diffusion scaling.

Reference notebooks and scripts for the book are available under `x_store/dawp/python36`.

The notebook is designed to run smoothly on Google Colab. A local setup is optional and
can be based on `python -m venv .venv`.

### 1. Environment check

We start by importing the core libraries for this class and by printing version
information to confirm that a modern Python environment is active.

In [ ]:
import sys  # access basic runtime information
from pathlib import Path  # path handling for optional figure export

import math  # elementary math functions

import numpy as np  # numerical arrays and linear algebra tools
import pandas as pd  # tabular data handling
import matplotlib as mpl  # matplotlib configuration
import matplotlib.pyplot as plt  # plotting

np.set_printoptions(precision=6, suppress=True)  # compact numeric output

print(sys.version.split()[0])  # Python version string
print("NumPy:", np.__version__)  # NumPy version string

FIG_SAVE = True  # set to True to export figures as PDFs
FIG_DIR = Path("../figures")  # figure output directory
FIG_DPI = 300  # target resolution for exported figures
FIG_DISPLAY = "svg"  # inline display format: "svg" or "png"

plt.style.use("seaborn-v0_8")  # readable plotting defaults
mpl.rcParams["figure.figsize"] = (8.0, 4.5)  # consistent figure size
mpl.rcParams["axes.grid"] = True  # show a grid for readability
mpl.rcParams["savefig.dpi"] = FIG_DPI  # default export resolution

try:
    from matplotlib_inline.backend_inline import (  # Jupyter helper
        set_matplotlib_formats,
    )
    set_matplotlib_formats(FIG_DISPLAY)  # configure inline plot rendering
except Exception:
    pass

if FIG_DISPLAY == "png":
    mpl.rcParams["figure.dpi"] = FIG_DPI  # high-resolution inline rendering


def maybe_save(fig, filename):
    # Optionally saves a Matplotlib figure as a PDF file.
    if not FIG_SAVE:
        return
    FIG_DIR.mkdir(parents=True, exist_ok=True)
    path = FIG_DIR / f"{filename}.pdf"
    fig.savefig(path, format="pdf", dpi=FIG_DPI)
    print(f"saved: {path}")

### 2. Complete market models

A market is called complete when every integrable payoff can be replicated by a
self-financing strategy. In such a setting, the equivalent martingale measure is unique
and arbitrage-free prices are uniquely determined.

- Completeness means replication is always possible.
- Uniqueness of the equivalent martingale measure implies a single arbitrage-free price.
- In practice, completeness is a modeling idealization that yields tractable analytics.
- The BSM model is the canonical continuous-time complete market model.
- The CRR model is a discrete-time complete market model that converges to BSM.

### 3. The Black&mdash;Scholes&mdash;Merton model

The BSM model combines a money-market account with a stock price modeled as geometric
Brownian motion. Under the risk-neutral measure, the drift of the stock becomes the
short rate, and the discounted stock price is a martingale.

### 4. BSM valuation formulas

For a European option with maturity $T$, strike $K$, constant short rate $r$, and constant
volatility $\sigma$, the BSM model leads to closed-form values.

For calls, the value is
$$C_0 = S_0 N(d_1) - e^{-rT} K N(d_2),$$
with
$$d_1 = \frac{\ln(S_0 / K) + (r + 0.5\sigma^2)T}{\sigma\sqrt{T}},\qquad d_2 = d_1 - \sigma\sqrt{T}.$$

For puts, put&mdash;call parity implies $P_0 = C_0 - S_0 + e^{-rT}K$ (no dividends).

In [ ]:
def norm_pdf(x):
    # Standard normal probability density function.
    return math.exp(-0.5 * x * x) / math.sqrt(2.0 * math.pi)


def norm_cdf(x):
    # Standard normal cumulative distribution function.
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))


def bsm_d1(S0, K, T, r, sigma):
    # Black-Scholes-Merton d1 term.
    num = math.log(S0 / K) + (r + 0.5 * sigma * sigma) * T
    den = sigma * math.sqrt(T)
    return num / den


def bsm_call_value(S0, K, T, r, sigma):
    # Black-Scholes-Merton European call option value (no dividends).
    if T <= 0.0:
        return max(S0 - K, 0.0)
    if sigma <= 0.0:
        return math.exp(-r * T) * max(S0 - K, 0.0)
    d1 = bsm_d1(S0, K, T, r, sigma)
    d2 = d1 - sigma * math.sqrt(T)
    return S0 * norm_cdf(d1) - math.exp(-r * T) * K * norm_cdf(d2)


def bsm_put_value(S0, K, T, r, sigma):
    # Black-Scholes-Merton European put option value via put-call parity.
    call = bsm_call_value(S0, K, T, r, sigma)
    return call - S0 + math.exp(-r * T) * K


S0 = 100.0  # initial underlying level
K = 100.0  # strike level
T = 1.0  # time to maturity in years
r = 0.05  # constant short rate
sigma = 0.20  # volatility

C0 = bsm_call_value(S0, K, T, r, sigma)  # call value
P0 = bsm_put_value(S0, K, T, r, sigma)  # put value

print(f"BSM call value: {C0:.6f}")
print(f"BSM put value:  {P0:.6f}")

### 5. BSM value curves

We now compute call and put values as functions of the underlying level and visualize the
results. The exported figure is used in the Part II slide deck.

In [ ]:
S0_grid = np.linspace(50.0, 150.0, 201)  # underlying grid for plotting
call_vals = np.array([bsm_call_value(x, K, T, r, sigma) for x in S0_grid])
put_vals = np.array([bsm_put_value(x, K, T, r, sigma) for x in S0_grid])

call_inner = np.maximum(S0_grid - K, 0.0)  # intrinsic call value
put_inner = np.maximum(K - S0_grid, 0.0)  # intrinsic put value

fig, ax = plt.subplots()  # create a single plot
ax.plot(S0_grid, call_vals, lw=2.2, label="call value")
ax.plot(S0_grid, put_vals, lw=2.2, label="put value")
ax.plot(S0_grid, call_inner, ls="--", lw=1.8, label="call intrinsic")
ax.plot(S0_grid, put_inner, ls="--", lw=1.8, label="put intrinsic")

ax.set_xlabel(r"underlying level $S_0$")  # x-axis label
ax.set_ylabel(r"value at $t=0$")  # y-axis label
ax.set_title("BSM call and put values")  # plot title
ax.legend(loc=0)  # place the legend

maybe_save(fig, "dawp_pII_fig01_bsm_values")  # optional PDF export
plt.show()  # display the plot

### 6. Greeks in the BSM model

Greeks are partial derivatives of the option value with respect to model inputs. They
provide local sensitivity measures used for hedging and risk reporting. We focus on delta,
gamma, and vega for a European call option.

In [ ]:
def bsm_call_delta(S0, K, T, r, sigma):
    # Delta of a European call option in the BSM model.
    d1 = bsm_d1(S0, K, T, r, sigma)
    return norm_cdf(d1)


def bsm_call_gamma(S0, K, T, r, sigma):
    # Gamma of a European call option in the BSM model.
    d1 = bsm_d1(S0, K, T, r, sigma)
    return norm_pdf(d1) / (S0 * sigma * math.sqrt(T))


def bsm_call_vega(S0, K, T, r, sigma):
    # Vega of a European call option in the BSM model.
    d1 = bsm_d1(S0, K, T, r, sigma)
    return S0 * norm_pdf(d1) * math.sqrt(T)


delta = np.array([bsm_call_delta(x, K, T, r, sigma) for x in S0_grid])
gamma = np.array([bsm_call_gamma(x, K, T, r, sigma) for x in S0_grid])
vega = np.array([bsm_call_vega(x, K, T, r, sigma) for x in S0_grid])

fig, axes = plt.subplots(3, 1, sharex=True)  # three-row figure

axes[0].plot(S0_grid, delta, lw=2.0)  # plot delta
axes[0].set_ylabel("delta")  # y-axis label

axes[1].plot(S0_grid, gamma, lw=2.0)  # plot gamma
axes[1].set_ylabel("gamma")  # y-axis label

axes[2].plot(S0_grid, vega, lw=2.0)  # plot vega
axes[2].set_ylabel("vega")  # y-axis label
axes[2].set_xlabel(r"underlying level $S_0$")  # x-axis label

fig.suptitle("BSM call Greeks (fixed K, T, r, sigma)")  # overall title
fig.tight_layout()  # improve spacing

maybe_save(fig, "dawp_pII_fig02_bsm_greeks")  # optional PDF export
plt.show()  # display the figure

### 7. Cox&mdash;Ross&mdash;Rubinstein model and convergence

The CRR model prices European options by backward induction on a binomial tree. Under the
diffusion scaling $u=\exp(\sigma\sqrt{\Delta t})$ and $d=1/u$, the CRR call value converges
to the BSM benchmark as the number of time steps increases.

In [ ]:
def crr_call_value(S0, K, T, r, sigma, M=100):
    # Cox-Ross-Rubinstein European call option value via backward induction.
    dt = T / M  # time step
    u = math.exp(sigma * math.sqrt(dt))  # up factor
    d = 1.0 / u  # down factor
    disc = math.exp(-r * dt)  # discount factor per step
    q = (math.exp(r * dt) - d) / (u - d)  # risk-neutral probability

    j = np.arange(M + 1)  # node index at maturity
    S_T = S0 * (u ** j) * (d ** (M - j))  # terminal stock levels
    V = np.maximum(S_T - K, 0.0)  # terminal call payoffs

    for _ in range(M, 0, -1):
        V = disc * (q * V[1:] + (1.0 - q) * V[:-1])  # backward step
    return float(V[0])


M_grid = np.arange(5, 401, 5)  # number of time steps for convergence plot
crr_vals = np.array([crr_call_value(S0, K, T, r, sigma, int(m)) for m in M_grid])

bsm_benchmark = bsm_call_value(S0, K, T, r, sigma)  # BSM benchmark call value

fig, ax = plt.subplots()  # create a single plot
ax.plot(M_grid, crr_vals, lw=2.0, label="CRR values")
ax.axhline(bsm_benchmark, ls="--", lw=2.0, color="C3", label="BSM benchmark")

ax.set_xlabel("number of binomial steps")  # x-axis label
ax.set_ylabel("European call value")  # y-axis label
ax.set_title("CRR convergence to the BSM benchmark")  # plot title
ax.legend(loc=0)  # place the legend

maybe_save(fig, "dawp_pII_fig03_crr_convergence")  # optional PDF export
plt.show()  # display the plot

### 8. Summary

Complete market models make risk-neutral valuation explicit. In the BSM model, European
option values and Greeks have closed forms. In the CRR model, the same no-arbitrage logic
is implemented by backward induction, and values converge to the BSM benchmark as the
tree is refined.

<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">